The goal of this notebook is to show the working of HyperKZG PCS, and how it is better than Gemini PCS, when it comes to MLE polynomials provided in evaluation format.


To run this notebook, you need to run the following commands:
 `conda create -n sage sage python=3.12.5`
    `conda activate sage`
    `sage -n jupyter`


In [1]:
# Define variables
n = 3
N = 2^n
X = var(['X{}'.format(i) for i in range(n)]) # Define the variables X_0, X-1,.....
u = var(['u{}'.format(i) for i in range(n)]) # Define the variables u_0, u-1,.....

# Function to get binary representation as a list of bits
def bits(i,n):
    return list(map(int, format(i,'0{}b'.format(n))))

def bits_reverse(i, n):
    bits_list = bits(i, n)
    return list(reversed(bits_list))

# Define the eq_tilde function
def eq_tilde(bits_i, u_vector):
    result=1
    for bit,u in zip(bits_i,u_vector):
        result *= (1-bit)*(1-u) + bit*u
    return result

# Coefficients of the polynomial
a = [var('a{}'.format(i)) for i in range(N)] # Coefficients a_0, a_1, ...., a_(N-1)

# MLE polynomial
f_tilde = sum(a[i]*eq_tilde(bits(i,n), X) for i in range(N))

for term in f_tilde.operands():
    print(term)

        
# Generate all combinations of (1-u[i]) and u[i] based on binary representation
def generate_c_vector(n, u):
    c_vector = []
    for i in range(2^n):  # Loop over all binary numbers from 0 to 2^n - 1
        binary = list(map(int, format(i, f'0{n}b')))  # Binary representation of i
        binary_reverse = list(reversed(binary))  # Reverse the binary representation
        product = 1
        for j, bit in enumerate(binary_reverse):
            if bit == 0:
                product *= (1 - u[j])  # Use (1 - u[j]) for 0
            else:
                product *= u[j]  # Use u[j] for 1
        c_vector.append(product)
    return c_vector

# Compute the c vector
c = generate_c_vector(n, u)

# Display the c vector
show(c)

# Create a 3D plot of f_tilde
# First, let's create a numerical version of f_tilde by substituting some values for coefficients
import numpy as np

# Set some random coefficients for visualization
coeffs = [1, 2, 3, 4, 5, 6, 7, 8]  # 8 coefficients for n=3

# Create a numerical function
def f_tilde_numerical(x0, x1, x2):
    result = 0
    for i in range(8):
        bits_i = bits(i, 3)
        result += coeffs[i] * eq_tilde(bits_i, [x0, x1, x2])
    return result

# Create points for all evaluations
points = []
for i in range(8):
    bits_i = bits(i, 3)
    x, y, z = bits_i
    points.append((x, y, z, coeffs[i]))

# Create point plot with axes labels
point_plot = point3d([(p[0], p[1], p[2]) for p in points], size=20, axes_labels=['X0', 'X1', 'X2'])

# Create text labels with offset, only showing coordinate and a value
text_plots = []
# offset = 0.22  # Spacing for clarity
# fontsize = 16  # Large and readable

for i, p in enumerate(points):
    label = f"({p[0]},{p[1]},{p[2]})\n" + f"a{i}={p[3]}"
    # If x=1, place label to the right; else to the left
    if p[0] == 1:
        label_pos = (p[0] , p[1], p[2])
    else:
        label_pos = (p[0] , p[1], p[2])
    text_plots.append(text3d(label, label_pos, fontsize=20))

# Combine all text labels into a single plot (no blue dots)
final_plot = sum(text_plots)
final_plot.show()

# --- Separate a-vector display section ---
a_vector_text = "a = [" + ", ".join([f"{c}" for c in coeffs]) + "]"
show(html(f"<pre style='font-size:14px'>{a_vector_text}</pre>"))

-(X0 - 1)*(X1 - 1)*(X2 - 1)*a0
(X0 - 1)*(X1 - 1)*X2*a1
(X0 - 1)*X1*(X2 - 1)*a2
-(X0 - 1)*X1*X2*a3
X0*(X1 - 1)*(X2 - 1)*a4
-X0*(X1 - 1)*X2*a5
-X0*X1*(X2 - 1)*a6
X0*X1*X2*a7


[-(u0 - 1)*(u1 - 1)*(u2 - 1),
 u0*(u1 - 1)*(u2 - 1),
 (u0 - 1)*u1*(u2 - 1),
 -u0*u1*(u2 - 1),
 (u0 - 1)*(u1 - 1)*u2,
 -u0*(u1 - 1)*u2,
 -(u0 - 1)*u1*u2,
 u0*u1*u2]

Graphics3d Object

<pre style='font-size:14px'>a = [1, 2, 3, 4, 5, 6, 7, 8]</pre>

So we have the evaluation vector with us which represents the evaluation of a boolean hypercube at individual points. For n=3, we have the points from `000` all the way up to `111`.

Now, in the Gemini Protocol, we take this evaluation vector, and convert this to the coefficient-value form using FFT/IFFT. 
The followind code snippet shows that

In [2]:
import sys
import os
!pip install -q py-ecc

src_path = os.path.abspath(os.path.join("..", "src"))
if src_path not in sys.path:
    sys.path.append(src_path)

# Define variables
n = 3
N = 2^n
X = var(['X{}'.format(i) for i in range(n)]) # Define the variables X_0, X-1,.....
u = var(['u{}'.format(i) for i in range(n)]) # Define the variables u_0, u-1,.....

# Function to get binary representation as a list of bits
def bits(i,n):
    return list(map(int, format(i,'0{}b'.format(n))))

def bits_reverse(i, n):
    bits_list = bits(i, n)
    return list(reversed(bits_list))

# Define the eq_tilde function
def eq_tilde(bits_i, u_vector):
    result=1
    for bit,u in zip(bits_i,u_vector):
        result *= (1-bit)*(1-u) + bit*u
    return result

def generate_c_vector(n, u):
    c_vector = []
    for i in range(2^n):  # Loop over all binary numbers from 0 to 2^n - 1
        binary = list(map(int, format(i, f'0{n}b')))  # Binary representation of i
        binary_reverse = list(reversed(binary))  # Reverse the binary representation
        product = 1
        for j, bit in enumerate(binary_reverse):
            if bit == 0:
                product *= (1 - u[j])  # Use (1 - u[j]) for 0
            else:
                product *= u[j]  # Use u[j] for 1
        c_vector.append(product)
    return c_vector

from mle2 import MLEPolynomial
from curve import Fr as BN254_Fr

MLEPolynomial.set_field_type(BN254_Fr)

evals = [1, 2, 3, 4, 5, 6, 7, 8]
print("Original evaluation at individual points")
print(f"evals = {[str(x) for x in evals]}\n")

evals = [BN254_Fr(int(x)) for x in evals]
coeffs = MLEPolynomial.compute_coeffs_from_evals(evals)
print("Coefficients after NTT:")
print(f"coeffs = {[str(x) for x in coeffs]}\n")

print("What does this mean?\n")

# Step 1: Setup for symbolic MLE polynomial
N = len(evals)
n = 3  # log2(N)
X = var(['X{}'.format(i) for i in range(n)])

print("MLE polynomial in evaluation form:\n")
for i in range(N):
    term = eq_tilde(bits(i, n), X)
    print(f"a[{i}] = {evals[i]} => term = {evals[i]}*({term})")

print("\ngets converted to the coefficient form, which is:\n")


print("Symbolic monomial basis (c_i * monomial):")
for i in range(len(coeffs)):
    i_bits = bits(i, n)
    monomial_terms = [f"X{j}" for j, b in enumerate(i_bits) if b == 1]
    monomial = "*".join(monomial_terms) if monomial_terms else "1"
    print(f"c{i} * {monomial}")


print("\nReal-valued coefficient vector:")
print(f"coeffs = {[str(c) for c in coeffs]}")


print("\nFull coefficient form (including zero coefficients):")
for i in range(len(coeffs)):
    mon_bits = bits(i, n)
    monomial = "*".join([f"X{j}" if b == 1 else "1" for j, b in enumerate(mon_bits)])
    coeff = coeffs[i]
    print(f"coeffs[{i}] = {coeff} => term = {coeff}*{monomial}")

print("\n\\ie. the coefficient form of the same multilinear extension")

Original evaluation at individual points
evals = ['1', '2', '3', '4', '5', '6', '7', '8']

Coefficients after NTT:
coeffs = ['1', '1', '2', '0', '4', '0', '0', '0']

What does this mean?

MLE polynomial in evaluation form:

a[0] = 1 => term = 1*(-(X0 - 1)*(X1 - 1)*(X2 - 1))
a[1] = 2 => term = 2*((X0 - 1)*(X1 - 1)*X2)
a[2] = 3 => term = 3*((X0 - 1)*X1*(X2 - 1))
a[3] = 4 => term = 4*(-(X0 - 1)*X1*X2)
a[4] = 5 => term = 5*(X0*(X1 - 1)*(X2 - 1))
a[5] = 6 => term = 6*(-X0*(X1 - 1)*X2)
a[6] = 7 => term = 7*(-X0*X1*(X2 - 1))
a[7] = 8 => term = 8*(X0*X1*X2)

gets converted to the coefficient form, which is:

Symbolic monomial basis (c_i * monomial):
c0 * 1
c1 * X2
c2 * X1
c3 * X1*X2
c4 * X0
c5 * X0*X2
c6 * X0*X1
c7 * X0*X1*X2

Real-valued coefficient vector:
coeffs = ['1', '1', '2', '0', '4', '0', '0', '0']

Full coefficient form (including zero coefficients):
coeffs[0] = 1 => term = 1*1*1*1
coeffs[1] = 1 => term = 1*1*1*X2
coeffs[2] = 2 => term = 2*1*X1*1
coeffs[3] = 0 => term = 0*1*X1*X2
coe

From the above code snippet, it can be seen that conversion from Evaluation vector to Coefficient Vector requires the use of `ntt_core` function which takes `O(NlogN)` time. 

The next step is mapping this is `coefficient vector` i.e. `['1', '1', '2', '0', '4', '0', '0', '0']` to a univariate polynomial and then use the `split-and-fold` technique thereafter. There is where the difference between Gemini and HyperKZG comes in. 

So, if we already have a coefficient form of a MLE-polynomial, we won't need the conversion from evaluation format. But what if we are presented with the evaluation form of the MLE polynomial. In case of Gemini, we will always be needing to first convert it to the coefficient form and then do the mapping. Instead, in the case of HyperKZG we do not need this conversion, therefore, be it evaluation form or coefficient form of the MLE, there is no effect on the time for proof generation.

Let's see how this works.

In [3]:
import sys
import os
!pip install -q py-ecc

# Define variables
n = 3
N = 2^n
X = var(['X{}'.format(i) for i in range(n)]) # Define the variables X_0, X-1,.....
u = var(['u{}'.format(i) for i in range(n)]) # Define the variables u_0, u-1,.....

# Function to get binary representation as a list of bits
def bits(i,n):
    return list(map(int, format(i,'0{}b'.format(n))))

def bits_reverse(i, n):
    bits_list = bits(i, n)
    return list(reversed(bits_list))

# Define the eq_tilde function
def eq_tilde(bits_i, u_vector):
    result=1
    for bit,u in zip(bits_i,u_vector):
        result *= (1-bit)*(1-u) + bit*u
    return result

from mle2 import MLEPolynomial
from curve import Fr as BN254_Fr

MLEPolynomial.set_field_type(BN254_Fr)
evals = [1, 2, 3, 4, 5, 6, 7, 8]  

evals = [BN254_Fr(int(x)) for x in evals] 
coeffs = MLEPolynomial.compute_coeffs_from_evals(evals)
# print("Coefficients after NTT:")


print("Gemini: Mapping the coefficient vector to a unvariate polynomial\n")

print(f"coeffs = {[str(x) for x in coeffs]}\n")

def uni_eval_from_coeffs(coeffs, z):
    return sum(int(coeffs[i]) * z**i for i in range(len(coeffs)))

x = var('X')  # Symbolic variable for univariate polynomial
univariate_poly_gemini = uni_eval_from_coeffs(coeffs, x)

# 1. Symbolic representation (using c0, c1, ..., c7)
symbolic_terms = [f"c{i}*X^{i}" for i in range(len(coeffs))]
symbolic_poly_str = " + ".join(symbolic_terms)
print("Symbolic coefficient form (before assigning actual values):")
print(symbolic_poly_str)

# 2. Actual coefficient vector
print("\nReal-valued coefficient vector:")
print(f"coeffs = {[str(x) for x in coeffs]}")

# 3. Final univariate polynomial in explicit form
terms = [f"{coeffs[i]}*X^{i}" for i in range(len(coeffs))]
poly_str = " + ".join(terms)
print("\nUnivariate polynomial (explicit form with zero coeffs):")
print(poly_str)

# 4. Evaluatable symbolic polynomial expression (optional, for display)
x = var('X')  # symbolic variable
univariate_poly_gemini = sum(int(coeffs[i]) * x**i for i in range(len(coeffs)))
print("\nSimplified symbolic expression (optional):")
print(univariate_poly_gemini)
print("\n")
print("HyperKZG: Mapping the evaluation vector to a unvariate polynomial\n")
print("Original evaluation at individual points")
print(f"evals = {[str(x) for x in evals]}\n")
univariate_poly_hyperKZG = uni_eval_from_coeffs(evals, x)
print(univariate_poly_hyperKZG)


Gemini: Mapping the coefficient vector to a unvariate polynomial

coeffs = ['1', '1', '2', '0', '4', '0', '0', '0']

Symbolic coefficient form (before assigning actual values):
c0*X^0 + c1*X^1 + c2*X^2 + c3*X^3 + c4*X^4 + c5*X^5 + c6*X^6 + c7*X^7

Real-valued coefficient vector:
coeffs = ['1', '1', '2', '0', '4', '0', '0', '0']

Univariate polynomial (explicit form with zero coeffs):
1*X^0 + 1*X^1 + 2*X^2 + 0*X^3 + 4*X^4 + 0*X^5 + 0*X^6 + 0*X^7

Simplified symbolic expression (optional):
4*X^4 + 2*X^2 + X + 1


HyperKZG: Mapping the evaluation vector to a unvariate polynomial

Original evaluation at individual points
evals = ['1', '2', '3', '4', '5', '6', '7', '8']

8*X^7 + 7*X^6 + 6*X^5 + 5*X^4 + 4*X^3 + 3*X^2 + 2*X + 1


In [4]:
import sys
import os
!pip install -q py-ecc

# Define variables
n = 3
N = 2^n
X = var(['X{}'.format(i) for i in range(n)]) # Define the variables X_0, X-1,.....
u = var(['u{}'.format(i) for i in range(n)]) # Define the variables u_0, u-1,.....

# Function to get binary representation as a list of bits
def bits(i,n):
    return list(map(int, format(i,'0{}b'.format(n))))

def bits_reverse(i, n):
    bits_list = bits(i, n)
    return list(reversed(bits_list))

# Define the eq_tilde function
def eq_tilde(bits_i, u_vector):
    result=1
    for bit,u in zip(bits_i,u_vector):
        result *= (1-bit)*(1-u) + bit*u
    return result

from mle2 import MLEPolynomial
from curve import Fr as BN254_Fr

MLEPolynomial.set_field_type(BN254_Fr)
evals = [1, 2, 3, 4, 5, 6, 7, 8]  

evals = [BN254_Fr(int(x)) for x in evals] 
coeffs = MLEPolynomial.compute_coeffs_from_evals(evals)
# print("Coefficients after NTT:")


print("Gemini: Mapping the coefficient vector to a unvariate polynomial\n")

print(f"coeffs = {[str(x) for x in coeffs]}\n")

def uni_eval_from_coeffs(coeffs, z):
    return sum(int(coeffs[i]) * z**i for i in range(len(coeffs)))

x = var('X')  # Symbolic variable for univariate polynomial
univariate_poly_gemini = uni_eval_from_coeffs(coeffs, x)

# 1. Symbolic representation (using c0, c1, ..., c7)
symbolic_terms = [f"c{i}*X^{i}" for i in range(len(coeffs))]
symbolic_poly_str = " + ".join(symbolic_terms)
print("Symbolic coefficient form (before assigning actual values):")
print(symbolic_poly_str)

# 2. Actual coefficient vector
print("\nReal-valued coefficient vector:")
print(f"coeffs = {[str(x) for x in coeffs]}")

# 3. Final univariate polynomial in explicit form
terms = [f"{coeffs[i]}*X^{i}" for i in range(len(coeffs))]
poly_str = " + ".join(terms)
print("\nUnivariate polynomial (explicit form with zero coeffs):")
print(poly_str)

# 4. Evaluatable symbolic polynomial expression (optional, for display)
x = var('X')  # symbolic variable
univariate_poly_gemini = sum(int(coeffs[i]) * x**i for i in range(len(coeffs)))
print("\nSimplified symbolic expression (optional):")
print(univariate_poly_gemini)
print("\n")
print("HyperKZG: Mapping the evaluation vector to a unvariate polynomial\n")
print("Original evaluation at individual points")
print(f"evals = {[str(x) for x in evals]}\n")
univariate_poly_hyperKZG = uni_eval_from_coeffs(evals, x)
print(univariate_poly_hyperKZG)


Gemini: Mapping the coefficient vector to a unvariate polynomial

coeffs = ['1', '1', '2', '0', '4', '0', '0', '0']

Symbolic coefficient form (before assigning actual values):
c0*X^0 + c1*X^1 + c2*X^2 + c3*X^3 + c4*X^4 + c5*X^5 + c6*X^6 + c7*X^7

Real-valued coefficient vector:
coeffs = ['1', '1', '2', '0', '4', '0', '0', '0']

Univariate polynomial (explicit form with zero coeffs):
1*X^0 + 1*X^1 + 2*X^2 + 0*X^3 + 4*X^4 + 0*X^5 + 0*X^6 + 0*X^7

Simplified symbolic expression (optional):
4*X^4 + 2*X^2 + X + 1


HyperKZG: Mapping the evaluation vector to a unvariate polynomial

Original evaluation at individual points
evals = ['1', '2', '3', '4', '5', '6', '7', '8']

8*X^7 + 7*X^6 + 6*X^5 + 5*X^4 + 4*X^3 + 3*X^2 + 2*X + 1


As it can be seen, we are mapping the coefficient vector to univariate polynomial for Gemini, and for HyperKZG, we use the evaluation vector i.e. `[1, 2, 3, 4, 5, 6, 7, 8]`

                                        --------------------------------------------
                ``HOW and WHY can we map the MLE Evaluation form and Coefficient form to a Univariate Polynomial ?``

In [5]:
## Explanation to be added soon

So we have both our polynomials in the univariate form, from Evaluation to UniPoly as well as from Coefficients to UniPoly. Next step is to show how we can reduce down these univariate polynomials through the split-and-fold technique.

Let's see how this works:

In [6]:
import sys
import os
!pip install -q py-ecc

# Define variables
n = 3
N = 2^n
X = var(['X{}'.format(i) for i in range(n)]) # Define the variables X_0, X-1,.....
u = var(['u{}'.format(i) for i in range(n)]) # Define the variables u_0, u-1,.....

# Function to get binary representation as a list of bits
def bits(i,n):
    return list(map(int, format(i,'0{}b'.format(n))))

def bits_reverse(i, n):
    bits_list = bits(i, n)
    return list(reversed(bits_list))

# Define the eq_tilde function
def eq_tilde(bits_i, u_vector):
    result=1
    for bit,u in zip(bits_i,u_vector):
        result *= (1-bit)*(1-u) + bit*u
    return result

from mle2 import MLEPolynomial
from curve import Fr as BN254_Fr
from hyperkzg_pcs import HYPERKZG_PCS                        
from unipoly import UniPolynomial
from curve import Fr as Field

print("\nCreating multilinear polynomial f and univariate polynomial g(X)...")

# 1) multilinear polynomial f  (from the evaluation vector we already have)
f_mle = MLEPolynomial(evals,3)                # BN254 field is already set

# 2) univariate poly g(X)  =  uₙ(evals)
g_uni = UniPolynomial(evals)                # coefficients are evals[i]

print("\nUnivariate poly via HYPERKZG map   g(X) =")
print(g_uni)              

x = var('X')                      # univariate indeterminate
def vec_poly(vec):                # vector → symbolic poly  P(X)=Σv[i]·X^i
    return sum(int(v) * x**i for i, v in enumerate(vec))

def official_fold_verbose(f_eval_vec, u_vec):
    print("\n_____ HyperKZG fold __________________________________")
    h = f_eval_vec[:]                                     # h⁰
    print("h⁰ =", [int(v) for v in h])

    for k, u_k in enumerate(u_vec):
        print(f"\nStep {k+1}:  u_{k} = {int(u_k)}")
        h_even = h[::2]
        h_odd  = h[1::2]
        print("  h_even =", [int(v) for v in h_even])
        print("  h_odd  =", [int(v) for v in h_odd])

        # build h_next element-by-element and show each computation
        h_next = []
        for idx, (a, b) in enumerate(zip(h_even, h_odd)):
            new_val = a + u_k * (b - a)
            h_next.append(new_val)
            print(f"   h_next[{idx}] = {int(a)} + {int(u_k)}·({int(b)}−{int(a)}) = {int(new_val)}")

        print("  h_next =", [int(v) for v in h_next])

        # Eq.(24) verification
        h_i_expr   = vec_poly(h)
        h_ip1_expr = vec_poly(h_next)
        h_i_neg    = h_i_expr.subs({x: -x})
        u_int      = int(u_k)
        rhs        = expand((1-u_int)*(h_i_expr + h_i_neg)/2 +
                            u_int     *(h_i_expr - h_i_neg)/(2*x))
        lhs        = h_ip1_expr.subs({x: x**2})
        print("  LHS = h^{%d}(X²) =" % (k+1), lhs)
        print("  RHS              =", rhs)
        print("  LHS − RHS        =", expand(lhs - rhs))

        h = h_next                                       # advance to next stage

    print("\nFinal folded constant =", int(h[0]))
    return h[0]

def bits_le(i, n=3):
    return [(i >> j) & 1 for j in range(n)]

def delta(bits_i, u_vec):
    one = Field(int(1))
    out = Field(int(1))
    for j, (bit, u) in enumerate(zip(bits_i, u_vec)):
        bit_f = Field(int(bit))
        term = (one - bit_f) * (one - u) + bit_f * u
        out  *= term
    return out


# choose challenge points
u_vals = [Field(int(v)) for v in (2, 5, 3)]
print("\nChallenge point vector u =", [int(v) for v in u_vals])

# run folding
fold_constant = official_fold_verbose(evals, u_vals)

# direct MLE evaluation 
print("\nComputing direct evaluation of f(u) using δ̃...")
direct = Field(int(0))
for i, e in enumerate(evals):
    b = bits_le(i)
    d = delta(b, u_vals)
    term = e * d
    direct += term
print(direct)

print("\n========= Final Check =========")
print("Constant after folds =", int(fold_constant))
print("Direct f(u)          =", int(direct))
print("Match?               →", fold_constant == direct)


Creating multilinear polynomial f and univariate polynomial g(X)...

Univariate poly via HYPERKZG map   g(X) =
1 + 2x + 3x^2 + 4x^3 + 5x^4 + 6x^5 + 7x^6 + 8x^7

Challenge point vector u = [2, 5, 3]

_____ HyperKZG fold __________________________________
h⁰ = [1, 2, 3, 4, 5, 6, 7, 8]

Step 1:  u_0 = 2
  h_even = [1, 3, 5, 7]
  h_odd  = [2, 4, 6, 8]
   h_next[0] = 1 + 2·(2−1) = 3
   h_next[1] = 3 + 2·(4−3) = 5
   h_next[2] = 5 + 2·(6−5) = 7
   h_next[3] = 7 + 2·(8−7) = 9
  h_next = [3, 5, 7, 9]
  LHS = h^{1}(X²) = 9*X^6 + 7*X^4 + 5*X^2 + 3
  RHS              = 9*X^6 + 7*X^4 + 5*X^2 + 3
  LHS − RHS        = 0

Step 2:  u_1 = 5
  h_even = [3, 7]
  h_odd  = [5, 9]
   h_next[0] = 3 + 5·(5−3) = 13
   h_next[1] = 7 + 5·(9−7) = 17
  h_next = [13, 17]
  LHS = h^{2}(X²) = 17*X^2 + 13
  RHS              = 17*X^2 + 13
  LHS − RHS        = 0

Step 3:  u_2 = 3
  h_even = [13]
  h_odd  = [17]
   h_next[0] = 13 + 3·(17−13) = 25
  h_next = [25]
  LHS = h^{3}(X²) = 25
  RHS              = 25
  LHS − RHS

In [7]:

import sys, os
!pip install -q py-ecc


from curve     import Fr  as Field
from mle2      import MLEPolynomial
from bcho_pcs  import BCHO_PCS           
from unipoly   import UniPolynomial
from curve     import Fr as BN254_Fr


n = 3
x = var('X')

def bits_le(i, n=n):
    """little-endian bits list"""
    return [(i >> j) & 1 for j in range(n)]

def vec_poly(vec):
    """vector → Σ vec[i]·X^i"""
    return sum(int(v)*x**i for i,v in enumerate(vec))

def delta(bits_i, u_vec):
    """δ̃_i(u) in the BN254 field"""
    one = Field(int(1))
    out = Field(int(1))
    for bit, u in zip(bits_i, u_vec):
        bit_f = Field(int(bit))
        out  *= (one-bit_f)*(one-u) + bit_f*u
    return out


evals  = [Field(int(v)) for v in (1,2,3,4,5,6,7,8)]
print("evaluations=",evals)

MLEPolynomial.set_field_type(BN254_Fr)
coeffs = MLEPolynomial.compute_coeffs_from_evals(evals)  
print("coeffs =", [int(c) for c in coeffs])

g_uni  = UniPolynomial(coeffs)                 
print("\nUnivariate poly P(X) =", g_uni)


def gemini_fold_verbose(c_vec, u_vec):
    print("\n________  Gemini split-and-fold  ________")
    c = c_vec[:]                                         
    print("c⁰ =", [int(v) for v in c])

    for k, u_k in enumerate(u_vec):
        print(f"\nStep {k+1}, u_{k} = {int(u_k)}")

        # split
        c_even, c_odd = c[::2], c[1::2]
        print("  c_even =", [int(v) for v in c_even])
        print("  c_odd  =", [int(v) for v in c_odd])

        # update  c_next[i] = c_even[i] + u·c_odd[i]
        c_next = []
        for idx, (a, b) in enumerate(zip(c_even, c_odd)):
            new = a + u_k * b
            c_next.append(new)
            print(f"   c_next[{idx}] = {int(a)} + {int(u_k)}·{int(b)} = {int(new)}")
        print("  c_next =", [int(v) for v in c_next])

        # ----- verify Eq.(13):  Q(X²) = (P+P(−X))/2 + u·(P−P(−X))/(2X)
        P      = vec_poly(c)          # current poly  P(X)
        P_neg  = P.subs({x: -x})
        Q      = vec_poly(c_next)     # poly after the fold
        u_int  = int(u_k)

        lhs = Q.subs({x: x**2})
        rhs = expand((P + P_neg)/Integer(2) +
                     u_int * (P - P_neg)/(Integer(2)*x))

        print("  LHS = Q(X²) =", lhs)
        print("  RHS         =", rhs)
        print("  LHS − RHS   =", expand(lhs - rhs))

        c = c_next                    # proceed to next round

    print("\nFinal folded constant =", int(c[0]))
    return c[0]    
u_vals = [Field(int(v)) for v in (2,5,3)]
print("evaluation point=",u_vals)
fold_const = gemini_fold_verbose(coeffs, u_vals)

# ----------------------------- direct evaluation check ---------------------------
direct = Field(int(0))
for i, e in enumerate(evals):
    b = bits_le(i)
    d = delta(b, u_vals)
    term = e * d
    direct += term
print(direct)

print("\n=========  Check  =========")
print("Constant after folds =", int(fold_const))
print("Direct f(u)          =", int(direct))
print("Match?               →", fold_const == direct)


evaluations= [1, 2, 3, 4, 5, 6, 7, 8]
coeffs = [1, 1, 2, 0, 4, 0, 0, 0]

Univariate poly P(X) = 1 + x + 2x^2 + 4x^4
evaluation point= [2, 5, 3]

________  Gemini split-and-fold  ________
c⁰ = [1, 1, 2, 0, 4, 0, 0, 0]

Step 1, u_0 = 2
  c_even = [1, 2, 4, 0]
  c_odd  = [1, 0, 0, 0]
   c_next[0] = 1 + 2·1 = 3
   c_next[1] = 2 + 2·0 = 2
   c_next[2] = 4 + 2·0 = 4
   c_next[3] = 0 + 2·0 = 0
  c_next = [3, 2, 4, 0]
  LHS = Q(X²) = 4*X^4 + 2*X^2 + 3
  RHS         = 4*X^4 + 2*X^2 + 3
  LHS − RHS   = 0

Step 2, u_1 = 5
  c_even = [3, 4]
  c_odd  = [2, 0]
   c_next[0] = 3 + 5·2 = 13
   c_next[1] = 4 + 5·0 = 4
  c_next = [13, 4]
  LHS = Q(X²) = 4*X^2 + 13
  RHS         = 4*X^2 + 13
  LHS − RHS   = 0

Step 3, u_2 = 3
  c_even = [13]
  c_odd  = [4]
   c_next[0] = 13 + 3·4 = 25
  c_next = [25]
  LHS = Q(X²) = 25
  RHS         = 25
  LHS − RHS   = 0

Final folded constant = 25
25

=========  Check  =========
Constant after folds = 25
Direct f(u)          = 25
Match?               → True


Now, finally we can have a look at the full commit-> prove-> verify process of HyperKZG. Let's see how that would look like:

In [8]:


from curve            import Fr  as Field, G1Point as G1, G2Point as G2
from kzg10_non_hiding import KZG10_PCS
from hyperkzg_pcs     import HYPERKZG_PCS
from mle2             import MLEPolynomial
from unipoly          import UniPolynomial
from merlin.merlin_transcript import MerlinTranscript


evals = [Field(v) for v in range(1, 9)]          # [1,2,3,4,5,6,7,8]
f      = MLEPolynomial(evals, 3)                 # multilinear poly
print(f"\n f.evals = {[int(e) for e in evals]}")


kzg = KZG10_PCS(G1, G2, Field, 20, debug=True)   # SRS supports deg ≤ 19
hyper = HYPERKZG_PCS(kzg, debug=True)


P_uni = UniPolynomial(f.evals)                   # P(X)=Σ evals[i] X^i
f_cm  = kzg.commit(P_uni)

print("\n    commitment C    =", f_cm)

# ---------------- transcript ----------------------------------------------------
tr = MerlinTranscript(b"demo-hyperkzg")
print(" transcript initialised")

# ---------------- evaluation point & expected value ----------------------------
u_point = [Field(int(2)), Field(int(5)), Field(int(3))]
value   = f.evaluate(u_point)
print("\n point u =", [int(u) for u in u_point])
print("    f(u)    =", int(value))

# ---------------- prover   ------------------------------------------------------
print("\n prover: split-and-fold + KZG opening (debug prints follow)")
val, proof = hyper.prove_eval(f_cm,
                              f,                # multilinear polynomial
                              u_point,
                              transcript = tr.fork(b"hyperkzg_pcs"),
                              debug      = 0)
print("\n prover returns")
print("    value =", int(val))
print("    proof =", proof)

# ---------------- verifier ------------------------------------------------------
print("\n verifier checking …")
ok = hyper.verify_eval(f_cm,
                       proof,
                       u_point,
                       val,
                       tr.fork(b"hyperkzg_pcs"),
                       debug = 0)
print("\n verifier verdict =", ok)
assert ok
print("\n✅ commit / prove / verify (n = 3, evals 1-8) succeeded")



 f.evals = [1, 2, 3, 4, 5, 6, 7, 8]

    commitment C    = Commitment((1506812733088703036033234108263636167068070115043288394654696824369162753701, 6768976303707139693812897756116472395786482991788971963782101192212253147496))
 transcript initialised

 point u = [2, 5, 3]
    f(u)    = 25

 prover: split-and-fold + KZG opening (debug prints follow)

 prover returns
    value = 25
    proof = {'h_poly_cm_vec': [Commitment((12639462353244010680381447368452078110804438495068713582928834100037754862265, 10866247309917626445353569018798471704498055674764137457556639609877370593858)), Commitment((4980580926989182459414445441806082896302700826507243621831503156722784301198, 14343466145928105863220159096908656068831682711784830237508107782651444368178))], 'evals_pos_beta': [2277209654400165544615964393156358858988247296874751437304457084, 5252628250030167080666728524, 14206714084], 'evals_neg_beta': [-2277209649631505683588547399815675656631316148717517213283000732, -525262824025290014041727